# RAG with Persona

## Welcome to this notebook!
RAG with personas was inspired by ....

## Installation

In [ ]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate
import cohere

### Connect to Weaviate Instance and Configure Model

In [ ]:
cohere_key = "API_KEY"

command_nightly = dspy.Cohere(model="command-nightly",max_tokens=1000, api_key=cohere_key)

weaviate_client = weaviate.Client("WEAVIATE_URL")
retriever_model = WeaviateRM("ChatMessage", weaviate_client=weaviate_client)
dspy.settings.configure(lm=command_nightly, rm=retriever_model)

## Configure Signature

In [ ]:
class AnswerWithPersona(dspy.Signature):
    """
    Respond to the chat messages with a response consistent with the given persona.
    """

    persona = dspy.InputField()
    message = dspy.InputField()
    message_contexts = dspy.InputField(desc="retrieved context relevant to the latest message.")
    response = dspy.OutputField()

## Build Program

In [ ]:
class RAGwithPersona(dspy.Module):
    def __init__(self):
        super().__init__() 
        self.retrieve = dspy.Retrieve(k=3)
        self.rag_with_persona = dspy.Predict(AnswerWithPersona)
    

    def forward(self, persona, message):
        message_contexts = self.retrieve(message).passages
        response = self.rag_with_persona(persona=persona, message=message, message_contexts=message_contexts).response
        return dspy.Prediction(response=response)

program = RAGwithPersona()

## Try it out

In [ ]:
program(persona="You are Taylor Swift", message="What will your new album be about?")

In [ ]:
command_nightly.inspect_history(n=1)

## Store response in the database

In [ ]:
from weaviate.util import get_valid_uuid
import uuid

user_chat_id = get_valid_uuid(uuid4())
chat_snippets.data.insert(
        properties={
            "author": "user",
            "content": parsed_history[-1]
        },
        uuid=user_chat_id
)